In [1]:
import urllib
import pandas as pd
import numpy as np
from datetime import date, timedelta
import requests
import boto3
import os
#from motionchart.motionchart import MotionChart, MotionChartDemo

In [2]:
#Getting the daily file if it exists
#url = "https://www.ecdc.europa.eu/sites/default/files/documents/_
#COVID-19-geographic-disbtribution-worldwide-"+ format(Sys.time(), "%Y-%m-%d"), ".xlsx", sep = "")
### 


In [3]:
#Define all the static entries
url= "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"
sheet_name="COVID-19-geographic-disbtributi"
#ouput file
S3_BUCKET_NAME=''

In [4]:
filedate=date.today().strftime('%Y-%m-%d')

# Get the file of today or yesterday
#Test if today's file exists

#Use yesterday's file if today is not uploaded
file_url=url+filedate+".xlsx"
request = requests.get(file_url)
if request.status_code != 200:
  filedate = (date.today()-timedelta(1)).strftime('%Y-%m-%d')
file_url=url+filedate+".xlsx"
coronafilename='CORONA'+filedate+'.xlsx'
urllib.request.urlretrieve(file_url, coronafilename) 
file_url

'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-21.xlsx'

In [5]:
df=pd.read_excel(io=coronafilename,sheet_name=sheet_name)

In [6]:
df['Day']=df['Day'].astype('category')
df['Month']=df['Month'].astype('category')
df['Year']=df['Year'].astype('category')
df['Countries and territories'] =df['Countries and territories'].astype('category')


In [7]:
df=df.sort_values(by='DateRep',ascending=True)

In [8]:
df['TotalCases']=0
df['TotalDeath']=0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6012 entries, 3005 to 6011
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateRep                    6012 non-null   datetime64[ns]
 1   Day                        6012 non-null   category      
 2   Month                      6012 non-null   category      
 3   Year                       6012 non-null   category      
 4   Cases                      6012 non-null   int64         
 5   Deaths                     6012 non-null   int64         
 6   Countries and territories  6012 non-null   category      
 7   GeoId                      6005 non-null   object        
 8   TotalCases                 6012 non-null   int64         
 9   TotalDeath                 6012 non-null   int64         
dtypes: category(4), datetime64[ns](1), int64(4), object(1)
memory usage: 366.3+ KB


In [20]:
df1=pd.DataFrame([])
df2=pd.DataFrame([])
for country in df['Countries and territories'].unique():
    df1=df[df['Countries and territories']==country]
    df1['TotalCases']=df[df['Countries and territories']==country]['Cases'].cumsum()
    df1['TotalDeath']=df[df['Countries and territories']==country]['Deaths'].cumsum()
    
    df2=pd.concat([df2,df1])
    
    #df1=pd.concat(df1, (df[df['Countries and territories']==country]['Cases']))
    #s=df[df['Countries and territories']==country]['Cases'].cumsum()
    #print(s)
    #df[df['Countries and territories']==country]['TotalCases']=s
df2['TotalDeath'].max()


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


4032

In [19]:
df2.tail()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,TotalCases,TotalDeath
3468,2020-03-21,21,3,2020,3,0,Madagascar,MG,3,0
4000,2020-03-21,21,3,2020,1,0,Niger,NE,1,0
3923,2020-03-21,21,3,2020,2,0,New_Caledonia,NC,2,0
3749,2020-03-21,21,3,2020,1,0,Montserrat,MS,1,0
6011,2020-03-21,21,3,2020,1,0,Zimbabwe,ZW,1,0


In [21]:
DAILY_FILE='CORONA'+filedate+'.csv'
df2.to_csv(DAILY_FILE)
print(DAILY_FILE)

CORONA2020-03-21.csv


In [22]:

#s3 = boto3.resource('s3')
#s3.meta.client.upload_file(DAILY_FILE, S3_BUCKET_NAME,DAILY_FILE)


In [23]:
f = open("coronafilename", "w")
f.write(DAILY_FILE)
f.close()

In [26]:
df2.tail(100)

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,TotalCases,TotalDeath
3127,2020-03-17,17,3,2020,0,0,Kosovo,XK,2,0
3126,2020-03-18,18,3,2020,17,0,Kosovo,XK,19,0
3125,2020-03-19,19,3,2020,0,0,Kosovo,XK,19,0
3124,2020-03-20,20,3,2020,2,0,Kosovo,XK,21,0
3123,2020-03-21,21,3,2020,3,0,Kosovo,XK,24,0
...,...,...,...,...,...,...,...,...,...,...
3468,2020-03-21,21,3,2020,3,0,Madagascar,MG,3,0
4000,2020-03-21,21,3,2020,1,0,Niger,NE,1,0
3923,2020-03-21,21,3,2020,2,0,New_Caledonia,NC,2,0
3749,2020-03-21,21,3,2020,1,0,Montserrat,MS,1,0
